# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

In [1]:
import backtrader as bt
import datetime
import pandas as pd

## Define Strategies and Data utils

In [2]:
class BasicStrategy(bt.Strategy):
    def __init__(self):
        # To keep track of pending orders
        self.order = None
        print('***The limit sells for this strategy are valid for 1 day***')
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

            # Note: This logic does not properly work
#         elif order.status in [order.Expired]:
#             if order.isbuy():
#                 self.log('BUY ORDER EXPIRED')
#             elif order.issell():
#                 self.log('SELL ORDER EXPIRED')
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        self.order = None



    def next(self):
        # STRATEGY 
        data = self.datas[0]
        self.log(f'Current Portfolio Value : {self.broker.get_value()}')
        
        # Check if we are in the market
        if not self.position:
            # BUY
            try:
                self.size = int(self.broker.get_cash() / self.datas[0].open[1])
            except:
                print('Size Exception. If at the end of data, ignore.')
            # invest if prediction looks good
            # TODO: Make this a filter in the class constructor
            if (data.prediction[0] >= data.close[0]):
                self.log(f'MARKET BUY CREATE {self.size} shares at next open, current close price: {data.close[0]}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(size=self.size) # market order buys at next open
                
        else:
            # place sell order at predicted high if predicted high is greater than current close price
            if data.prediction[0] > data.close[0]:
                self.log(f'LIMIT SELL CREATE {self.size} shares at {data.prediction[0]}')
                # Keep track of the created order to avoid a 2nd order, order valid for 1 day
                self.order = self.sell(exectype=bt.Order.Limit,
                             price=data.prediction[0],
                             valid=data.datetime.date(0) + datetime.timedelta(days=2),
                             size=self.size)
            # if prediction is less than current value sell at open
            else:
                self.log('MARKET SELL CREATE. PREDICTION < CURRENT CLOSE')
                self.order = self.sell(size=self.size)
        

In [3]:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

      
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.datas[0].open[1])
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print(f'Stop price: {self.datas[0].close[0]}')
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [4]:
def prepare_data(data, fromdate, todate, filepath):
    """Prepare data for backtrader datafeed object
        Returns prepared data filepath and params for the GenericCSVData class, also returns columns used"""
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    # start param setup for backtrader
    start = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    from_to = [(start.year, start.month, start.day), (end.year, end.month, end.day)]
    # Backtrader string format
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # backtrader data feed class needs a file path, this could be a temp file that's constantly overwritten
    data.to_csv(filepath)
    
    starting_params = [
    ('fromdate', datetime.datetime(*from_to[0])),
    ('todate', datetime.datetime(*from_to[1])),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d %H:%M:%S')),
    ('tmformat', ('%H:%M:%S')),
    ('datetime', 1)]
    # skip nonfeatures (timestamp)
    cols = data.columns[1:]
    # get column position for each indicator and add to starting params list
    i = 2 # starting index since others are reserved
    for indicator in cols:
        starting_params.append((indicator, i))
        i+=1
    final_params = tuple(starting_params)

    return filepath, final_params, cols 


## Import Data and Run Backtest

In [5]:
import os

In [6]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../data/ticker_predictions/{pred_folder_name}'):
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df['prediction'] = pred_df[pred_col_name]
        # remove all non-prediction rows, use "timestamp" as the common datetime name
        ticker_df = ticker_df[ticker_df.prediction > 0]
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)

        # make timestamp in column position 0 
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']

        # skip nonfeatures, keep timestamp
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]

        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        ##### BUY AND HOLD #######
        print('BUY AND HOLD')
        # initialize everything and run strategy
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        #cerebro.addsizer(bt.sizers.FixedSize, stake=10)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BuyAndHold)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        buy_hold = cerebro.run()
        buy_hold_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(buy_hold[0].analyzers.sharpe.get_analysis())
        print(buy_hold[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(buy_hold[0].analyzers.returns.get_analysis())

        #### MAIN STRATEGY #####
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BasicStrategy)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        back = cerebro.run()
        strat_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(back[0].analyzers.sharpe.get_analysis())
        print(back[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(back[0].analyzers.returns.get_analysis())
        print(f"For {ticker_name}, return was {buy_hold_ret*100}% for buy and hold vs {strat_ret*100}%  for strategy")
        print('\n'*5)

        ticker_backtesting_dict[ticker_name] = cerebro
        
    return ticker_backtesting_dict
#     return ticker_df

In [7]:
# pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000

cerebro_dict = capstone_data_prep('rfreg_preds', 'prediction', '../data/ticker_data')
cerebro_dict

--------Ticker Name-------: WAT
BUY AND HOLD
Starting Portfolio Value: 100000
2000-04-03, BUY EXECUTED, 47.25
Stop price: 24.950000762939453
ROI:        -47.19%
Final Portfolio Value: 52813.20161437988

STRATEGY INFO:
OrderedDict([('sharperatio', -0.05310412646377665)])
AutoOrderedDict([('len', 417), ('drawdown', 72.38561503019314), ('moneydown', 138439.29838562012), ('max', AutoOrderedDict([('len', 417), ('drawdown', 74.93031505846456), ('moneydown', 143306.10080718994)]))])
OrderedDict([('rtot', -0.6384089959529341), ('ravg', -0.001175707174867282), ('rnorm', -0.2564194708602946), ('rnorm100', -25.64194708602946)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2000-03-31, Current Portfolio Value : 100000.0
2000-03-31, MARKET BUY CREATE 2116 shares at next open, current close price: 47.625
2000-04-03, BUY EXECUTED, 47.25
2000-04-03, Current Portfolio Value : 92197.25
2000-04-03, LIMIT SELL CREATE 2116 shares at 230.7246024439876
2000-04-0

2000-09-28, Current Portfolio Value : 191252.5
2000-09-28, LIMIT SELL CREATE 2116 shares at 181.9738122246077
2000-09-29, Current Portfolio Value : 188343.0
2000-09-29, LIMIT SELL CREATE 2116 shares at 190.7136614210964
2000-10-02, Current Portfolio Value : 187285.0
2000-10-02, LIMIT SELL CREATE 2116 shares at 185.20900160346017
2000-10-03, Current Portfolio Value : 184375.5
2000-10-03, LIMIT SELL CREATE 2116 shares at 185.78829438251103
2000-10-04, Current Portfolio Value : 159777.0
2000-10-04, LIMIT SELL CREATE 2116 shares at 181.73942611271912
2000-10-05, Current Portfolio Value : 151313.0
2000-10-05, LIMIT SELL CREATE 2116 shares at 170.6713002546658
2000-10-06, Current Portfolio Value : 154487.0
2000-10-06, LIMIT SELL CREATE 2116 shares at 188.3460069769121
2000-10-09, Current Portfolio Value : 150784.0
2000-10-09, LIMIT SELL CREATE 2116 shares at 184.52949483187257
2000-10-10, Current Portfolio Value : 150651.75
2000-10-10, LIMIT SELL CREATE 2116 shares at 198.95405690312563
2000

2001-03-30, Current Portfolio Value : 98307.20161437988
2001-03-30, LIMIT SELL CREATE 2116 shares at 207.19823760435213
2001-04-02, Current Portfolio Value : 90753.08226013184
2001-04-02, LIMIT SELL CREATE 2116 shares at 207.5435138298986
2001-04-03, Current Portfolio Value : 86775.0
2001-04-03, LIMIT SELL CREATE 2116 shares at 208.5344498718816
2001-04-04, Current Portfolio Value : 83410.55967712402
2001-04-04, LIMIT SELL CREATE 2116 shares at 205.7286769348032
2001-04-05, Current Portfolio Value : 90478.0
2001-04-05, LIMIT SELL CREATE 2116 shares at 197.4793123611846
2001-04-06, Current Portfolio Value : 90795.40322875977
2001-04-06, LIMIT SELL CREATE 2116 shares at 196.33714065645265
2001-04-09, Current Portfolio Value : 92445.88064575195
2001-04-09, LIMIT SELL CREATE 2116 shares at 196.3411708467309
2001-04-10, Current Portfolio Value : 81485.0
2001-04-10, LIMIT SELL CREATE 2116 shares at 192.8091964560708
2001-04-11, Current Portfolio Value : 78543.7612915039
2001-04-11, LIMIT SEL

2001-09-25, Current Portfolio Value : 66249.79838562012
2001-09-25, LIMIT SELL CREATE 2116 shares at 268.8740641849931
2001-09-26, Current Portfolio Value : 71963.0
2001-09-26, LIMIT SELL CREATE 2116 shares at 267.0879054426571
2001-09-27, Current Portfolio Value : 74925.40322875977
2001-09-27, LIMIT SELL CREATE 2116 shares at 275.1558697296102
2001-09-28, Current Portfolio Value : 75708.32096862793
2001-09-28, LIMIT SELL CREATE 2116 shares at 274.9146700080037
2001-10-01, Current Portfolio Value : 73211.44032287598
2001-10-01, LIMIT SELL CREATE 2116 shares at 275.5447771388337
2001-10-02, Current Portfolio Value : 72386.20161437988
2001-10-02, LIMIT SELL CREATE 2116 shares at 271.26607141472067
2001-10-03, Current Portfolio Value : 76216.15644836426
2001-10-03, LIMIT SELL CREATE 2116 shares at 275.0758565928411
2001-10-04, Current Portfolio Value : 77253.0
2001-10-04, LIMIT SELL CREATE 2116 shares at 272.19437167517344
2001-10-05, Current Portfolio Value : 75877.59677124023
2001-10-05

2002-03-19, Current Portfolio Value : 63901.04113006592
2002-03-19, LIMIT SELL CREATE 2116 shares at 410.6640968046092
2002-03-20, Current Portfolio Value : 65720.79838562012
2002-03-20, LIMIT SELL CREATE 2116 shares at 410.9162855117129
2002-03-21, Current Portfolio Value : 61806.20161437988
2002-03-21, LIMIT SELL CREATE 2116 shares at 410.9162855117129
2002-03-22, Current Portfolio Value : 61594.60080718994
2002-03-22, LIMIT SELL CREATE 2116 shares at 416.6762669622301
2002-03-25, Current Portfolio Value : 61002.11935424805
2002-03-25, LIMIT SELL CREATE 2116 shares at 417.6809312500434
2002-03-26, Current Portfolio Value : 60303.839515686035
2002-03-26, LIMIT SELL CREATE 2116 shares at 419.763054274399
2002-03-27, Current Portfolio Value : 59034.2387084961
2002-03-27, LIMIT SELL CREATE 2116 shares at 424.9379714697275
2002-03-28, Current Portfolio Value : 59203.5185470581
2002-03-28, LIMIT SELL CREATE 2116 shares at 427.1067881938684
2002-04-01, Current Portfolio Value : 59055.399192

...

2000-10-13, Current Portfolio Value : 53645.65625
2000-10-13, LIMIT SELL CREATE 9922 shares at 187.2824925725384
2000-10-16, Current Portfolio Value : 52250.375
2000-10-16, LIMIT SELL CREATE 9922 shares at 176.60242140939445
2000-10-17, Current Portfolio Value : 49769.875
2000-10-17, LIMIT SELL CREATE 9922 shares at 163.41112315944582
2000-10-18, Current Portfolio Value : 49614.84375
2000-10-18, LIMIT SELL CREATE 9922 shares at 168.18107171879703
2000-10-19, Current Portfolio Value : 51630.25
2000-10-19, LIMIT SELL CREATE 9922 shares at 158.33531044712913
2000-10-20, Current Portfolio Value : 53335.59375
2000-10-20, LIMIT SELL CREATE 9922 shares at 140.37340974832424
2000-10-23, Current Portfolio Value : 53025.53125
2000-10-23, LIMIT SELL CREATE 9922 shares at 161.64026043845547
2000-10-24, Current Portfolio Value : 52870.5
2000-10-24, LIMIT SELL CREATE 9922 shares at 151.74418378148067
2000-10-25, Current Portfolio Value : 53490.625
2000-10-25, LIMIT SELL CREATE 9922 shares at 158.492

2001-04-10, Current Portfolio Value : 74221.40393924713
2001-04-10, LIMIT SELL CREATE 9922 shares at 246.0294341090625
2001-04-11, Current Portfolio Value : 74419.84375
2001-04-11, LIMIT SELL CREATE 9922 shares at 250.339329797656
2001-04-12, Current Portfolio Value : 77446.05204677582
2001-04-12, LIMIT SELL CREATE 9922 shares at 248.64924408427584
2001-04-16, Current Portfolio Value : 77222.80666828156
2001-04-16, LIMIT SELL CREATE 9922 shares at 256.40507462291146
2001-04-17, Current Portfolio Value : 80422.64920806885
2001-04-17, LIMIT SELL CREATE 9922 shares at 264.3587205380235
2001-04-18, Current Portfolio Value : 86822.34375
2001-04-18, LIMIT SELL CREATE 9922 shares at 245.22470139577047
2001-04-19, Current Portfolio Value : 92502.69102096558
2001-04-19, LIMIT SELL CREATE 9922 shares at 237.23088530025103
2001-04-20, Current Portfolio Value : 91237.63072204591
2001-04-20, LIMIT SELL CREATE 9922 shares at 228.0412173196712
2001-04-23, Current Portfolio Value : 85284.43639945984
2

2001-10-03, Current Portfolio Value : 86723.12147903442
2001-10-03, LIMIT SELL CREATE 9922 shares at 306.21749862830103
2001-10-04, Current Portfolio Value : 84341.84375
2001-10-04, LIMIT SELL CREATE 9922 shares at 304.42775994863564
2001-10-05, Current Portfolio Value : 83647.3067779541
2001-10-05, LIMIT SELL CREATE 9922 shares at 309.1294173102567
2001-10-08, Current Portfolio Value : 84143.39920806885
2001-10-08, LIMIT SELL CREATE 9922 shares at 301.47866072191897
2001-10-09, Current Portfolio Value : 83498.47337150574
2001-10-09, LIMIT SELL CREATE 9922 shares at 294.04398623999674
2001-10-10, Current Portfolio Value : 86400.66329193115
2001-10-10, LIMIT SELL CREATE 9922 shares at 302.78816154456626
2001-10-11, Current Portfolio Value : 90146.21412849426
2001-10-11, LIMIT SELL CREATE 9922 shares at 288.08227887890234
2001-10-12, Current Portfolio Value : 88682.71875
2001-10-12, LIMIT SELL CREATE 9922 shares at 279.9394439273896
2001-10-15, Current Portfolio Value : 88310.63996505737

2002-03-25, Current Portfolio Value : 112793.18177795409
2002-03-25, LIMIT SELL CREATE 9922 shares at 332.649231278212
2002-03-26, Current Portfolio Value : 112669.14920806885
2002-03-26, LIMIT SELL CREATE 9922 shares at 336.60041882562945
2002-03-27, Current Portfolio Value : 114281.47799301149
2002-03-27, LIMIT SELL CREATE 9922 shares at 336.6078727557905
2002-03-28, Current Portfolio Value : 115819.38534355164
2002-03-28, LIMIT SELL CREATE 9922 shares at 335.13787019389656
2002-04-01, Current Portfolio Value : 115521.72799301149
2002-04-01, LIMIT SELL CREATE 9922 shares at 332.53879642826297
2002-04-02, Current Portfolio Value : 112545.12610054016
2002-04-02, LIMIT SELL CREATE 9922 shares at 335.9093873714055
2002-04-03, Current Portfolio Value : 107658.54753494264
2002-04-03, LIMIT SELL CREATE 9922 shares at 336.65533821835686
2002-04-04, Current Portfolio Value : 108129.83912849426
2002-04-04, LIMIT SELL CREATE 9922 shares at 334.0373708661665
2002-04-05, Current Portfolio Value :

{'WAT': <backtrader.cerebro.Cerebro at 0x7fe978b70410>,
 'UNH': <backtrader.cerebro.Cerebro at 0x7fe990fcae90>,
 'HD': <backtrader.cerebro.Cerebro at 0x7fe978b12990>,
 'XOM': <backtrader.cerebro.Cerebro at 0x7fe991962350>,
 'MSFT': <backtrader.cerebro.Cerebro at 0x7fe980cffe50>,
 'ADSK': <backtrader.cerebro.Cerebro at 0x7fe9816f9a50>}

### Library additionally has plotting
Not sure how to reformat this, it's pretty ugly

In [22]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12}) 
cerebro_dict['UNH'].plot()

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/opt/anaconda3/lib/python3.7/site-packages/matplotlib/dates.py)

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open. Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements.  